In [1]:
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil

sys.path.insert(0, "\\".join(os.path.abspath(os.curdir).split("\\")[:-2:]))

from src.utils.utils import *

## 1. MammoTherm

In [ ]:
folders_names = ['Cisto', 'Lesao Benigna', 'Lesao Maligna', 'Sem lesao']
path = 'D:\\mathe\\Documents\\BancoDados_IC\\Termografia\\MammoTherm'

metadata = {'patientId': [], 'imagePath': [], 'pathology': [], 'orientation': []}

for folder_name in folders_names:
    folder_path = path + '\\' + folder_name + '\\'
    directory = Path(folder_path)
    images_path = list(directory.rglob("*.jpg*"))
    
    for image_path in images_path:
        image_name = str(image_path).split("\\")[-1].replace('.jpg', '')
        metadata['patientId'].append(image_name)
        metadata['imagePath'].append(str(image_path))
        metadata['pathology'].append(folder_name.replace('Lesao ', '').replace('Sem lesao', 'Normal'))
        
        if 'LEMD' in image_name: metadata['orientation'].append('LEMD')
        elif 'LEME' in image_name: metadata['orientation'].append('LEME')
        elif 'LIMD' in image_name: metadata['orientation'].append('LIMD')
        elif 'LIME' in image_name: metadata['orientation'].append('LIME')
        elif 'MD' in image_name: metadata['orientation'].append('MD')
        elif 'ME' in image_name: metadata['orientation'].append('ME')
        elif 'T1' in image_name: metadata['orientation'].append('T1')
        elif 'T2' in image_name: metadata['orientation'].append('T2')
        
metadata = pd.DataFrame(metadata)
images_size = get_images_size("D:\\mathe\\Documents\\BancoDados_IC\\Termografia\\MammoTherm", "jpg", True)
metadata['imageSizeMb'] = images_size

metadata.to_csv('../../outputs/termografia/Mammotherm_metadata.csv', index=False)
metadata.to_csv('D:\\mathe\\Documents\\BancoDados_IC\\Termografia\\Mammotherm_metadata.csv', index=False)

## DMRI

In [9]:
path = 'D:\\mathe\\Documents\\BancoDados_IC\\Termografia\\DMRI'
directory = Path(path)
images_path = list(directory.rglob("*.txt*"))
rois_path = list(directory.rglob("*.png*"))

metadata = {'studyId': [], 'patientId': [], 'imagePath': [], 'pathology': [], 'croppedImagePath': []}

for image_path in images_path:
    image_name = str(image_path).split("\\")[-1].replace('.txt', '')
    study_id = str(image_path).split("\\")[-3]
    
    metadata['studyId'].append(study_id)
    metadata['patientId'].append(image_name)
    
    img = []
    folder_path = f'{path}_processado/{study_id}/original'
    
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    if f'{image_name}.png' not in str(os.listdir(folder_path)):
        with open(image_path, 'r') as fd:
            for line in fd:
                tmp = line.replace("\t", " ").replace(",", ".").split(" ")
                tmp = [float(i) for i in tmp]
                img.append(tmp)
            
            img = np.array(img) 

            plt.imshow(img, cmap='jet')
            plt.xticks([])
            plt.yticks([])

            plt.savefig(folder_path + f'/{image_name}.png', bbox_inches='tight', pad_inches=0)
        #print(study_id)
        
    metadata['imagePath'].append(folder_path + f'/{image_name}.png')
    
    if 'DOENTES' in str(image_path):
        metadata['pathology'].append('Doente')
    else:
        metadata['pathology'].append('Normal')
    
    has_roi = False
    for roi in rois_path:
        if image_name in str(roi):
            roi_name = str(roi).split("\\")[-1]
            roi_folder_path = folder_path.replace("original", "cropped")
            
            if not os.path.exists(roi_folder_path):
                os.makedirs(roi_folder_path)
            
            if f'{roi_name}' not in os.listdir(roi_folder_path):
                shutil.copy(str(roi), roi_folder_path + f'/{roi_name}')
            metadata['croppedImagePath'].append(roi_folder_path + f'/{roi_name}')
            has_roi = True
            break

    if not has_roi:
        metadata['croppedImagePath'].append(None)
        
metadata = pd.DataFrame(metadata)

images_size = []
for folder in os.listdir('D:\\mathe\\Documents\\BancoDados_IC\\Termografia\\DMRI_processado'):
    path_folder = 'D:\\mathe\\Documents\\BancoDados_IC\\Termografia\\DMRI_processado\\' + folder + '\\original'
    images_size += get_images_size(path_folder, "png", True)
metadata['imageSizeMb'] = images_size

metadata.to_csv('../../outputs/termografia/DMRI_metadata.csv', index=False)
metadata.to_csv('D:\\mathe\\Documents\\BancoDados_IC\\Termografia\\DMRI_metadata.csv', index=False)